In [1]:
import time
import numpy as np
from src.data_loader import load_data
from src.feature_engineering import average_daily
from src.model.train_model import get_X_y, train_and_evaluate_models, get_best_fold_data_from_kf
import os
from objective_function import objective_et, objective_lasso
from src.Optimiser.KOA.koa_optimizer import koa_optimizer
from src.Optimiser.COA.coa_optimizer import coa_optimizer
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from analysis.wilcoxon_test import perform_wilcoxon_test

DATA_PATH = os.path.join("data", "Data.xlsx")


In [2]:
print("🔹 Loading raw data...")
df = load_data(DATA_PATH)
print(f"✅ Raw data shape: {df.shape}")

🔹 Loading raw data...
✅ Raw data shape: (50000, 12)


In [3]:
print("🔹 Aggregating daily averages (96 samples per day)...")
df_avg = average_daily(df, samples_per_day=96)
print(f"✅ Aggregated data shape: {df_avg.shape}")
print("✅ Sample preview:")
print(df_avg.head())

🔹 Aggregating daily averages (96 samples per day)...
✅ Aggregated data shape: (520, 12)
✅ Sample preview:
   Voltage (V)  Current (A)  Power Consumption (kW)  Temperature (Â°C)  \
0   229.442065    23.274581                5.338170          24.458245   
1   230.252406    27.590016                6.355897          25.281132   
2   230.139098    26.616651                6.113400          25.286834   
3   230.450687    28.109668                6.483036          26.025139   
4   230.131332    26.532084                6.102769          24.729869   

   Humidity (%)  Reactive Power (kVAR)  Power Factor  Solar Power (kW)  \
0     49.931362               1.335934      0.904576         24.128362   
1     47.592609               1.547024      0.902831         25.205873   
2     46.656764               1.560915      0.898813         22.725012   
3     51.258872               1.580661      0.905492         24.628754   
4     49.909403               1.526028      0.902028         23.925562   

   W

In [4]:
print("🔹 Preparing features and target...")
X, y = get_X_y(df_avg)
print(f"✅ X shape: {X.shape}, y shape: {y.shape}")


🔹 Preparing features and target...
✅ X shape: (520, 11), y shape: (520,)


In [5]:
from src.Utils.K_Fold import K_Fold


print("🔹 Performing K-Fold...")
(
    X_train,
    X_test,
    y_train,
    y_test,
    etr_scores,
    lasso_scores,
    combined_df,
) = K_Fold(X, y, n_splits=5)

🔹 Performing K-Fold...

🔁 Fold 1 ------------------
  ETR → R2: 0.9690, RMSE: 0.0031, MAE: 0.0409, MAPE: 0.65%, MARD: 0.33%
  Lasso → R2: 0.9251, RMSE: 0.0075, MAE: 0.0676, MAPE: 1.08%, MARD: 0.54%

🔁 Fold 2 ------------------
  ETR → R2: 0.9749, RMSE: 0.0021, MAE: 0.0353, MAPE: 0.56%, MARD: 0.28%
  Lasso → R2: 0.9306, RMSE: 0.0059, MAE: 0.0589, MAPE: 0.93%, MARD: 0.46%

🔁 Fold 3 ------------------
  ETR → R2: 0.9646, RMSE: 0.0036, MAE: 0.0421, MAPE: 0.66%, MARD: 0.33%
  Lasso → R2: 0.9249, RMSE: 0.0076, MAE: 0.0677, MAPE: 1.07%, MARD: 0.54%

🔁 Fold 4 ------------------
  ETR → R2: 0.9667, RMSE: 0.0037, MAE: 0.0500, MAPE: 0.81%, MARD: 0.40%
  Lasso → R2: 0.9174, RMSE: 0.0093, MAE: 0.0795, MAPE: 1.28%, MARD: 0.64%

🔁 Fold 5 ------------------
  ETR → R2: 0.9725, RMSE: 0.0024, MAE: 0.0386, MAPE: 0.61%, MARD: 0.30%
  Lasso → R2: 0.9305, RMSE: 0.0060, MAE: 0.0620, MAPE: 0.98%, MARD: 0.49%
ETR and Lasso folds are identical. Using shared variables.
✅ Combined DataFrame using original target 

# add to excel file 

In [6]:
# import pandas as pd
# from openpyxl import load_workbook

# # Load workbook
# book = load_workbook(DATA_PATH)

# # If the sheet exists, delete it
# if 'DATA after K-Fold' in book.sheetnames:
#     std = book['DATA after K-Fold']
#     book.remove(std)
#     book.save(DATA_PATH)

# # Now write the new data
# with pd.ExcelWriter(DATA_PATH, engine='openpyxl', mode='a') as writer:
#     combined_df.to_excel(writer, sheet_name='DATA after K-Fold', index=False)


In [7]:
def summarize_metrics(metrics_list):
  return {key: np.mean([m[key] for m in metrics_list]) for key in metrics_list[0]}

avg_etr = summarize_metrics(etr_scores)
avg_lasso = summarize_metrics(lasso_scores)

print("\n🔹 Average Baseline Model Scores:")
print("🔹 Extra Trees Regressor:")
for k, v in avg_etr.items():
  print(f"  {k}: {v:.4f}")

print("\n🔹 Lasso Regression:")
for k, v in avg_lasso.items():
  print(f"  {k}: {v:.4f}")



🔹 Average Baseline Model Scores:
🔹 Extra Trees Regressor:
  R2: 0.9695
  RMSE: 0.0030
  MAE: 0.0414
  MAPE: 0.6588
  MARD: 0.3290

🔹 Lasso Regression:
  R2: 0.9257
  RMSE: 0.0073
  MAE: 0.0672
  MAPE: 1.0667
  MARD: 0.5327


# Define the parameter bounds for the optimizers


In [8]:

#Extra Trees Regressor
lb_et = [10, 1, 2, 1]          # n_estimators, max_depth, min_samples_split, min_samples_leaf
ub_et = [200, 50, 10, 10]
dim_et = 4

# Lasso Regression
lb_lasso = [0.1, 100]       # alpha, max_iter
ub_lasso = [0.3, 1000]
dim_lasso = 2

n_agents = 3
max_iter = 5

# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [9]:
X, y = get_X_y(combined_df)

In [10]:
from src.model.train_model import train_raw_model

etr_result = train_raw_model("ETR", X_train, y_train, X_test, y_test)
lr_result = train_raw_model("LR", X_train, y_train, X_test, y_test)

# Access results
print("🔹 Extra Trees MAE:", etr_result["mae"])
print("🔹 Lasso Regression MAE:", lr_result["mae"])
print("🔧 Best Params (ETR):", etr_result["params"])
print("🔧 Best Params (LR):", lr_result["params"])



🔹 Extra Trees MAE: 0.03523802916026711
🔹 Lasso Regression MAE: 0.05407689392706924
🔧 Best Params (ETR): {'random_state': 42, 'n_estimators': 99, 'max_depth': 3, 'min_samples_split': 5}
🔧 Best Params (LR): {'random_state': 42, 'alpha': 0.1, 'max_iter': 1000, 'tol': 0.01}


In [11]:
from src.Optimiser.KOA.koa_optimizer import koa_optimizer 

In [12]:
from objective_function import objective_et, objective_lasso

In [13]:
best_pos_et, best_fit_et = koa_optimizer(
    objective_et, lb_et, ub_et, dim_et, n_agents, max_iter, X_train, y_train, X_test, y_test
)

🔁 Iter 1/5 - Best MAE: 0.01821
🔁 Iter 2/5 - Best MAE: 0.01821
🔁 Iter 3/5 - Best MAE: 0.01821
🔁 Iter 4/5 - Best MAE: 0.01821
🔁 Iter 5/5 - Best MAE: 0.01821


In [14]:

best_pos_lasso, best_fit_lasso = koa_optimizer(
    objective_lasso, lb_lasso, ub_lasso, dim_lasso, n_agents, max_iter, X_train, y_train, X_test, y_test
)

🔁 Iter 1/5 - Best MAE: 0.05859
🔁 Iter 2/5 - Best MAE: 0.05859
🔁 Iter 3/5 - Best MAE: 0.05859
🔁 Iter 4/5 - Best MAE: 0.05859
🔁 Iter 5/5 - Best MAE: 0.05859


In [15]:
# coa_optimizer
from src.Optimiser.COA.coa_optimizer import coa_optimizer

In [16]:

#Extra Trees Regressor
lb_et = [2, 2, 10, 4]
ub_et = [200, 100, 100, 10]  # or whatever int upper limit you want     
dim_et = 4


n_agents = 10
max_iter = 20

best_pos_et, best_fit_et = coa_optimizer(
    objective_et, lb_et, ub_et, dim_et, n_agents, max_iter, X_train, y_train, X_test, y_test
)


🦞 Iter 1/20 - Best MAE: 0.01847
🦞 Iter 2/20 - Best MAE: 0.01813
🦞 Iter 3/20 - Best MAE: 0.01813
🦞 Iter 4/20 - Best MAE: 0.01813
🦞 Iter 5/20 - Best MAE: 0.01813
🦞 Iter 6/20 - Best MAE: 0.01813
🦞 Iter 7/20 - Best MAE: 0.01813
🦞 Iter 8/20 - Best MAE: 0.01813
🦞 Iter 9/20 - Best MAE: 0.01813
🦞 Iter 10/20 - Best MAE: 0.01813
🦞 Iter 11/20 - Best MAE: 0.01810
🦞 Iter 12/20 - Best MAE: 0.01810
🦞 Iter 13/20 - Best MAE: 0.01810
🦞 Iter 14/20 - Best MAE: 0.01810
🦞 Iter 15/20 - Best MAE: 0.01810
🦞 Iter 16/20 - Best MAE: 0.01810
🦞 Iter 17/20 - Best MAE: 0.01810
🦞 Iter 18/20 - Best MAE: 0.01810
🦞 Iter 19/20 - Best MAE: 0.01810
🦞 Iter 20/20 - Best MAE: 0.01810


In [25]:
# Lasso Regression
lb_lasso = [0.1, 1000]       # alpha, max_iter
ub_lasso = [1, 2000]
dim_lasso = 2
n_agents = 100
max_iter = 20

best_pos_lasso, best_fit_lasso = coa_optimizer(
    objective_lasso, lb_lasso, ub_lasso, dim_lasso, n_agents, max_iter, X_train, y_train, X_test, y_test
)

🦞 Iter 1/20 - Best MAE: 0.05408
🦞 Iter 2/20 - Best MAE: 0.05408
🦞 Iter 3/20 - Best MAE: 0.05408
🦞 Iter 4/20 - Best MAE: 0.05408
🦞 Iter 5/20 - Best MAE: 0.05408
🦞 Iter 6/20 - Best MAE: 0.05408
🦞 Iter 7/20 - Best MAE: 0.05408
🦞 Iter 8/20 - Best MAE: 0.05408
🦞 Iter 9/20 - Best MAE: 0.05408
🦞 Iter 10/20 - Best MAE: 0.05408
🦞 Iter 11/20 - Best MAE: 0.05408
🦞 Iter 12/20 - Best MAE: 0.05408
🦞 Iter 13/20 - Best MAE: 0.05408
🦞 Iter 14/20 - Best MAE: 0.05408
🦞 Iter 15/20 - Best MAE: 0.05408
🦞 Iter 16/20 - Best MAE: 0.05408
🦞 Iter 17/20 - Best MAE: 0.05408
🦞 Iter 18/20 - Best MAE: 0.05408
🦞 Iter 19/20 - Best MAE: 0.05408
🦞 Iter 20/20 - Best MAE: 0.05408
